In [1]:
import pandas as pd
from pulp import *
import warnings
warnings.filterwarnings('ignore')

### Import Parameters

#### Nutrition Facts

In [2]:
nutrition = pd.read_excel('Nutrition Facts.xlsx', index_col = 0)
nutrition

,Protein,Fat,Fibre,Salt,Sugar
Ingredients,,,,,
Chicken,0.100000,0.080000,0.001000,0.002,0.000
Beef,0.200000,0.100000,0.005000,0.005,0.000
Mutton,0.150000,0.110000,0.003000,0.007,0.000
Rice,0.000000,0.010000,0.100000,0.002,0.000
Wheat bran,0.040000,0.010000,0.150000,0.008,0.000
Corn,0.032927,0.012805,0.028049,0.000,0.045
Peanuts,0.258000,0.492000,0.085000,0.001,0.047


#### Ingredients Costs

In [3]:
costs = pd.read_excel('Costs.xlsx')
costs

,Ingredients,Costs
0,Chicken,0.095
1,Beef,0.150
2,Mutton,0.100
3,Rice,0.002
4,Wheat bran,0.005
5,Corn,0.012
6,Peanuts,0.013


In [4]:
dict_costs = dict(zip(costs['Ingredients'], costs['Costs']))
dict_costs

{'Chicken': 0.095,
 'Beef': 0.15,
 'Mutton': 0.1,
 'Rice': 0.002,
 'Wheat bran': 0.005,
 'Corn': 0.012,
 'Peanuts': 0.013}

### Model Building
The objective is to minimize the cost of our bar, hence we will apply LpMinimize. <br>Because we can't have negative value of ingredient quantity, hence the low bound is zero.

In [5]:
# variables
variables = ['Chicken', 'Beef', 'Mutton', 'Rice', 'Wheat bran', 'Corn', 'Peanuts']

# initialize class
model = LpProblem("Optimize your Protein Bar", LpMinimize)

# Create Decision Variables
x = LpVariable.dicts("Qty", [j for j in variables], 
                     lowBound = 0, upBound = None, cat = 'continuous')


### Define the objective and add constraints

In [6]:
# Define Objective Function
model += (lpSum([dict_costs[i] * x[i] for i in variables]))

# Add Constraints 
model += (lpSum([x[i] for i in variables])) == 120
model += (lpSum([x[i] * nutrition.loc[i, 'Protein'] for i in variables])) >= 22
model += (lpSum([x[i] * nutrition.loc[i, 'Fat'] for i in variables])) <= 22
model += (lpSum([x[i] * nutrition.loc[i, 'Fibre'] for i in variables])) >= 6
model += (lpSum([x[i] * nutrition.loc[i, 'Salt'] for i in variables])) <= 3
model += (lpSum([x[i] * nutrition.loc[i, 'Sugar'] for i in variables])) <= 20

### Solve model and analyse

In [7]:
model.solve()
print("Cost per Bar = ${:,}".format(round(value(model.objective), 2)))
print('\n' + "Status: {}".format(LpStatus[model.status]))
for v in model.variables():
    print(v.name, "=", round(v.varValue, 2), 'g')

Cost per Bar = $10.32

Status: Optimal
Qty_Beef = 65.32 g
Qty_Chicken = 0.0 g
Qty_Corn = 0.0 g
Qty_Mutton = 0.0 g
Qty_Peanuts = 30.96 g
Qty_Rice = 0.0 g
Qty_Wheat_bran = 23.72 g


### Modifications (Infinite possibilities)
#### What if your customers want corn?
You can add corn to this recipe by adding a constraint of a minimum quantity of corn.
#### Do we have infinite possibilities?
Let us try to change the quantity from 120g to 100g. What is the result?

In [8]:
# Define Objective Function
model += (lpSum([dict_costs[i] * x[i] for i in variables]))

# Add Constraints 
model += (lpSum([x[i] for i in variables])) == 100
model += (lpSum([x['Corn']])) == 15
model += (lpSum([x[i] * nutrition.loc[i, 'Protein'] for i in variables])) >= 22
model += (lpSum([x[i] * nutrition.loc[i, 'Fat'] for i in variables])) <= 22
model += (lpSum([x[i] * nutrition.loc[i, 'Fibre'] for i in variables])) >= 6
model += (lpSum([x[i] * nutrition.loc[i, 'Salt'] for i in variables])) <= 3
model += (lpSum([x[i] * nutrition.loc[i, 'Sugar'] for i in variables])) <= 20

In [9]:
model.solve()
print("Cost per Bar = ${:,}".format(round(value(model.objective), 2)))
print('\n' + "Status: {}".format(LpStatus[model.status]))
for v in model.variables():
    print(v.name, "=", round(v.varValue, 2), 'g')

Cost per Bar = $11.08

Status: Infeasible
Qty_Beef = 71.27 g
Qty_Chicken = 0.0 g
Qty_Corn = 0.0 g
Qty_Mutton = 0.0 g
Qty_Peanuts = 30.26 g
Qty_Rice = 0.0 g
Qty_Wheat_bran = -1.53 g


> There is no optimal solution for the above combination of constraints. 